In [18]:
!nvidia-smi

Thu Feb 12 07:59:33 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   72C    P0             30W /   70W |    2951MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
!pip install ultralytics

In [2]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.8/112.6 GB disk)


In [12]:
# !rm -rf Fire-and-smoke-detection-1


In [ ]:
!pip install roboflow

from roboflow import Roboflow

rf = Roboflow(api_key="")
project = rf.workspace("alpaca-u9b8a").project("fire-and-smoke-detection-iaiyy-d862n")
version = project.version(1)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Fire-and-smoke-detection-1 in yolov8:: 100%|██████████| 47944/47944 [00:13<00:00, 3547.23it/s]


In [14]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

In [15]:
model.info()

YOLOv8n summary: 129 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs


(129, 3157200, 0, 8.8575488)

In [ ]:
results = model.train(
    data="/content/Fire-and-smoke-detection-1/data.yaml",
    epochs=60,
    imgsz=512,
    batch=32,
    patience=15,
    workers=4,
    cache=True
)

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Fire-and-smoke-detection-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=1

In [ ]:
from google.colab import files
files.download("runs/detect/train/weights/best.pt")

In [ ]:
metrics = model.val()
print(metrics)

Testing on img

In [ ]:
# model = YOLO("runs/detect/train/weights/best.pt")

# results = model.predict("some_test_image.jpg", save=True, show=True)

#Main

In [ ]:
from ultralytics import YOLO
import datetime

model = YOLO("runs/detect/train/weights/best.pt")

def detect_fire(image_path):
    results = model(image_path)

    fire_events = []

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            confidence = float(box.conf[0])
            cls = int(box.cls[0])

            area_pixels = (x2 - x1) * (y2 - y1)

            fire_events.append({
                "latitude": 28.6139,   # simulate for hackathon
                "longitude": 77.2090,
                "confidence": confidence,
                "area_pixels": area_pixels,
                "type": "fire" if cls == 0 else "smoke",
                "timestamp": str(datetime.datetime.now())
            })

    return fire_events